In [1]:
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    RigidTransform,
    RotationMatrix,
    Sphere,
    Box,
    RollPitchYaw,
)
from pydrake.geometry import Rgba
from manipulation.station import LoadScenario, MakeHardwareStation

In [3]:
meshcat = StartMeshcat()
print(f"Meshcat: {meshcat.web_url()}")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Meshcat: http://localhost:7001


In [4]:
scenario_yaml = """
directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://drake_models/wsg_50_description/sdf/schunk_wsg_50_welded_fingers.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
      translation: [0, 0, 0.09]
      rotation: !Rpy { deg: [90, 0, 90] }
"""

scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)

builder = DiagramBuilder()
builder.AddSystem(station)
diagram = builder.Build()

print("Scene built (minimal version)")

Scene built (minimal version)


In [5]:
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
context = simulator.get_mutable_context()

# Get plant from station
plant = station.GetSubsystemByName("plant")
plant_context = plant.GetMyContextFromRoot(context)

# Publish initial state
simulator.AdvanceTo(0.1)

print("Ready - robot arm loaded")

Ready - robot arm loaded


In [6]:
bin_x, bin_y = 0, -0.6
bin_w, bin_h, bin_d = 0.25, 0.25, 0.08
wall_thickness = 0.01

# Bin walls (visual only in Meshcat)
bin_color = Rgba(0.6, 0.4, 0.3, 0.6)
meshcat.SetObject("bin/floor", Box(bin_w, bin_h, wall_thickness), bin_color)
meshcat.SetTransform("bin/floor", RigidTransform([bin_x, bin_y, 0.0]))

meshcat.SetObject("bin/front", Box(bin_w, wall_thickness, bin_d), bin_color)
meshcat.SetTransform("bin/front", RigidTransform([bin_x, bin_y - bin_h/2, bin_d/2]))

meshcat.SetObject("bin/back", Box(bin_w, wall_thickness, bin_d), bin_color)
meshcat.SetTransform("bin/back", RigidTransform([bin_x, bin_y + bin_h/2, bin_d/2]))

meshcat.SetObject("bin/left", Box(wall_thickness, bin_h, bin_d), bin_color)
meshcat.SetTransform("bin/left", RigidTransform([bin_x - bin_w/2, bin_y, bin_d/2]))

meshcat.SetObject("bin/right", Box(wall_thickness, bin_h, bin_d), bin_color)
meshcat.SetTransform("bin/right", RigidTransform([bin_x + bin_w/2, bin_y, bin_d/2]))

# 2 balls
ball_radius = 0.05
blue_positions = [[-0.05, -0.65, 0.04]]

for i, pos in enumerate(blue_positions):
    meshcat.SetObject(f"balls/blue_{i}", Sphere(ball_radius), Rgba(0.2, 0.4, 0.9, 1.0))
    meshcat.SetTransform(f"balls/blue_{i}", RigidTransform(pos))

# Red target ball
red_pos = np.array([0.05, -0.55, 0.04])
meshcat.SetObject("balls/red_target", Sphere(ball_radius), Rgba(0.9, 0.1, 0.1, 1.0))
meshcat.SetTransform("balls/red_target", RigidTransform(red_pos))

In [7]:
# helper functions
import time
from pydrake.all import InverseKinematics, Solve

def compute_sphere_grasp(ball_center, ball_radius):
    """Compute top-down grasp pose for sphere"""
    finger_depth = 0.01
    grasp_height = ball_radius + finger_depth
    p_WG = ball_center + np.array([0, 0, grasp_height])
    R_WG = RotationMatrix(RollPitchYaw([np.pi, 0, 0]))
    X_WG = RigidTransform(R_WG, p_WG)
    
    return X_WG

def solve_ik_position_priority(plant, plant_context, target_position, 
                                 orientation_target=None, pos_tol=0.005):
    """IK solver prioritizing position over strict orientation"""
    ik = InverseKinematics(plant, plant_context)
    gripper_frame = plant.GetFrameByName("body")
    
    ik.AddPositionConstraint(
        gripper_frame, [0, 0, 0], plant.world_frame(),
        target_position - pos_tol, target_position + pos_tol
    )
    
    if orientation_target is not None:
        if isinstance(orientation_target, RigidTransform):
            orientation_target = orientation_target.rotation()
        ik.AddOrientationConstraint(
            gripper_frame, RotationMatrix(), plant.world_frame(),
            orientation_target, 0.3
        )
    
    q_nominal = plant.GetPositions(plant_context, plant.GetModelInstanceByName("iiwa"))
    # change later
    ik.prog().AddQuadraticErrorCost(np.eye(7) * 5.0, q_nominal, ik.q())
    
    result = Solve(ik.prog())
    if result.is_success():
        return True, result.GetSolution(ik.q())
    return False, None

def simulate_point_cloud_from_scene():
    """Simulate point cloud with colors from scene"""
    points, colors = [], []
    
    # Blue balls
    blue_rgb = np.array([0.2, 0.4, 0.9])
    for pos in blue_positions:
        for _ in range(100):
            offset = np.random.randn(3)
            offset = offset / np.linalg.norm(offset) * ball_radius
            point = np.array(pos) + offset * np.random.uniform(0.8, 1.0)
            points.append(point)
            colors.append(blue_rgb + np.random.randn(3) * 0.05)
    
    # Red ball
    red_rgb = np.array([0.9, 0.1, 0.1])
    for _ in range(100):
        offset = np.random.randn(3)
        offset = offset / np.linalg.norm(offset) * ball_radius
        point = red_pos + offset * np.random.uniform(0.8, 1.0)
        points.append(point)
        colors.append(red_rgb + np.random.randn(3) * 0.05)
    
    return np.array(points), np.array(colors)

def color_segment_red(points, colors):
    """Filter red points from point cloud"""
    colors = colors.astype(float)
    is_red = (colors[:, 0] > 0.5) & (colors[:, 1] < 0.3) & (colors[:, 2] < 0.3)
    return points[is_red]

def find_ball_center_from_points(points):
    """Compute centroid of points"""
    if len(points) == 0:
        return None
    return np.mean(points, axis=0)

In [8]:
# Final pick sequence manipulation
import time

print("=== Complete pick sequence with RECORDING ===\n")

# Define home configuration (neutral pose for IIWA)
q_home = np.array([0, 0.3, 0, -1.5, 0, 1.2, 0])

# Move to home position first
print("Step 0: Moving to HOME position...")
plant.SetPositions(plant_context, plant.GetModelInstanceByName("iiwa"), q_home)
diagram.ForcedPublish(context)
time.sleep(0.5)
print("  ✓ At home - ready to plan\n")

# Compute grasp pose for the red ball (using red_pos directly)
print("Computing grasp pose for red ball...")
X_WG_grasp = compute_sphere_grasp(red_pos, ball_radius)
print(f"  ✓ Grasp pose computed for ball at {red_pos}\n")

# solve all IK from this consistent starting point
print("Planning all waypoints from home configuration:")

# Waypoint 1: Pre-grasp (15cm above ball)
pre_grasp_height = 0.15
pre_grasp_pos = red_pos + np.array([0, 0, pre_grasp_height])
success_pre, q_pregrasp = solve_ik_position_priority(
    plant, plant_context, pre_grasp_pos, 
    orientation_target=X_WG_grasp.rotation(), pos_tol=0.005
)

# Waypoint 2: Grasp (at ball)
grasp_pos = X_WG_grasp.translation()
success_grasp, q_grasp = solve_ik_position_priority(
    plant, plant_context, grasp_pos, 
    orientation_target=X_WG_grasp.rotation(), pos_tol=0.003
)

# Waypoint 3: Lift (20cm above ball)
lift_height = 0.20
lift_pos = red_pos + np.array([0, 0, lift_height])
success_lift, q_lift = solve_ik_position_priority(
    plant, plant_context, lift_pos,
    orientation_target=X_WG_grasp.rotation(), pos_tol=0.005
)

# Waypoint 4: Pre-throw (lift + 15cm up)
prethrow_height_offset = 0.15
prethrow_pos = lift_pos + np.array([0, 0, prethrow_height_offset])
success_prethrow, q_prethrow = solve_ik_position_priority(
    plant, plant_context, prethrow_pos,
    orientation_target=X_WG_grasp.rotation(), pos_tol=0.01
)

# Build waypoint list
waypoints = []
waypoint_names = []
successes = [success_pre, success_grasp, success_lift, success_prethrow]
names = ["Pre-grasp", "Grasp", "Lift", "Pre-throw"]
qs = [q_pregrasp, q_grasp, q_lift, q_prethrow]
positions = [pre_grasp_pos, grasp_pos, lift_pos, prethrow_pos]

for success, name, q, pos in zip(successes, names, qs, positions):
    status = "✓" if success else "✗"
    print(f"{status} {name} at {pos}")
    if success:
        waypoints.append(q)
        waypoint_names.append(name)

print(f"\n✓ Successfully planned {len(waypoints)}/4 waypoints\n")

# Visualize waypoints
if success_pre:
    meshcat.SetObject("waypoints/pregrasp", Sphere(0.02), Rgba(1, 1, 0, 0.6))
    meshcat.SetTransform("waypoints/pregrasp", RigidTransform(pre_grasp_pos))
if success_grasp:
    meshcat.SetObject("waypoints/grasp", Sphere(0.02), Rgba(0, 1, 0, 0.8))
    meshcat.SetTransform("waypoints/grasp", RigidTransform(grasp_pos))
if success_lift:
    meshcat.SetObject("waypoints/lift", Sphere(0.02), Rgba(0, 1, 1, 0.6))
    meshcat.SetTransform("waypoints/lift", RigidTransform(lift_pos))
if success_prethrow:
    meshcat.SetObject("waypoints/prethrow", Sphere(0.025), Rgba(1, 0.65, 0, 1.0))
    meshcat.SetTransform("waypoints/prethrow", RigidTransform(prethrow_pos))

# Execute full sequence if we have all 4 waypoints
if len(waypoints) == 4:
    print("=== Executing full sequence: Home → Pre-grasp → Grasp → Lift → Pre-throw ===\n")
    
    meshcat.StartRecording()
    print("Recording started\n")
    
    all_waypoints = [q_home] + waypoints
    all_names = ["Home"] + waypoint_names
    
    sim_time = 0.0
    dt = 0.03
    
    for i in range(len(all_waypoints) - 1):
        q_start = all_waypoints[i]
        q_end = all_waypoints[i + 1]
        name = all_names[i + 1]
        
        print(f"  → Moving to {name}...")
        
        num_steps = 35
        for step in range(num_steps + 1):
            alpha = step / num_steps
            q_interp = (1 - alpha) * q_start + alpha * q_end
            
            plant.SetPositions(plant_context, plant.GetModelInstanceByName("iiwa"), q_interp)
            context.SetTime(sim_time)
            diagram.ForcedPublish(context)
            
            time.sleep(dt)
            sim_time += dt
        
        print(f"    ✓ {name}")
        time.sleep(0.4)
        sim_time += 0.4
    
    meshcat.StopRecording()
    print("\n🎬 Recording stopped\n")
    
    meshcat.PublishRecording()
    
    print("="*60)
    print("SUCCESS! Complete pick sequence executed")
    print("="*60)
    print("Robot is at PRE-THROW position")
    print("Ball is grasped and elevated")
    print(f"   - Total duration: {sim_time:.1f}s")
    print("="*60)
    
elif len(waypoints) >= 3:
    print("Only 3/4 waypoints succeeded - executing partial sequence")
    meshcat.StartRecording()
    
    all_waypoints = [q_home] + waypoints
    all_names = ["Home"] + waypoint_names
    sim_time = 0.0
    dt = 0.03
    
    for i in range(len(all_waypoints) - 1):
        q_start = all_waypoints[i]
        q_end = all_waypoints[i + 1]
        name = all_names[i + 1]
        
        print(f"  → Moving to {name}...")
        num_steps = 35
        for step in range(num_steps + 1):
            alpha = step / num_steps
            q_interp = (1 - alpha) * q_start + alpha * q_end
            plant.SetPositions(plant_context, plant.GetModelInstanceByName("iiwa"), q_interp)
            context.SetTime(sim_time)
            diagram.ForcedPublish(context)
            time.sleep(dt)
            sim_time += dt
        time.sleep(0.4)
        sim_time += 0.4
    
    meshcat.StopRecording()
    meshcat.PublishRecording()
    print("Partial sequence recorded")
else:
    print("Too few waypoints succeeded - check IK solutions")

=== Complete pick sequence with RECORDING ===

Step 0: Moving to HOME position...
  ✓ At home - ready to plan

Computing grasp pose for red ball...
  ✓ Grasp pose computed for ball at [ 0.05 -0.55  0.04]

Planning all waypoints from home configuration:
✓ Pre-grasp at [ 0.05 -0.55  0.19]
✓ Grasp at [ 0.05 -0.55  0.1 ]
✓ Lift at [ 0.05 -0.55  0.24]
✓ Pre-throw at [ 0.05 -0.55  0.39]

✓ Successfully planned 4/4 waypoints

=== Executing full sequence: Home → Pre-grasp → Grasp → Lift → Pre-throw ===

Recording started

  → Moving to Pre-grasp...
    ✓ Pre-grasp
  → Moving to Grasp...
    ✓ Grasp
  → Moving to Lift...
    ✓ Lift
  → Moving to Pre-throw...
    ✓ Pre-throw

🎬 Recording stopped

SUCCESS! Complete pick sequence executed
Robot is at PRE-THROW position
Ball is grasped and elevated
   - Total duration: 5.9s
